# Phase 1: Data Cleaning & Ground Truth Generation

## Objectives
1. Load 8 years of BTC hourly OHLCV data from Binance
2. Handle missing hours using domain-appropriate interpolation
3. Implement Garman-Klass volatility estimator
4. Create forward-shifted target variable for next-hour prediction
5. Validate data quality and save cleaned dataset

---

## 1. Setup & Data Loading

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 6)

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✓ Libraries imported successfully")

In [ ]:
# Load dataset
DATA_FILE = Path('btc_1h_data_2018_to_2025.csv')
print(f"Loading data from: {DATA_FILE}")

df = pd.read_csv(DATA_FILE)

# Display basic info
print(f"\n✓ Loaded {len(df):,} hourly records")
print(f"✓ Date range: {df.iloc[0]['timestamp'] if 'timestamp' in df.columns else 'N/A'} to {df.iloc[-1]['timestamp'] if 'timestamp' in df.columns else 'N/A'}")
print(f"\nDataset shape: {df.shape}")
print(f"\nColumn names:\n{df.columns.tolist()}")

In [ ]:
# Preview first few rows
print("First 10 rows of raw data:")
df.head(10)

In [ ]:
# Data types and missing values summary
print("Data Types:")
print(df.dtypes)
print("\n" + "="*60)
print("Missing Values Summary:")
missing_summary = pd.DataFrame({
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2)
})
print(missing_summary[missing_summary['Missing_Count'] > 0])

if missing_summary['Missing_Count'].sum() == 0:
    print("\n✓ No missing values detected!")
else:
    print(f"\n⚠ Total missing values: {missing_summary['Missing_Count'].sum():,}")

---
## 2. Data Cleaning & Missing Value Handling

In [ ]:
# Standardize column names (lowercase)
df.columns = df.columns.str.lower()

# Parse timestamp if exists
if 'timestamp' in df.columns:
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values('timestamp').reset_index(drop=True)
    print("✓ Timestamp parsed and sorted")
elif 'date' in df.columns:
    df['timestamp'] = pd.to_datetime(df['date'])
    df = df.sort_values('timestamp').reset_index(drop=True)
    print("✓ Date column converted to timestamp and sorted")
else:
    print("⚠ No timestamp column found - assuming data is already sorted chronologically")

print(f"\nData spans from {df['timestamp'].min()} to {df['timestamp'].max()}" if 'timestamp' in df.columns else "")

In [ ]:
# Check for temporal gaps (missing hours)
if 'timestamp' in df.columns:
    # Create expected hourly range
    expected_range = pd.date_range(
        start=df['timestamp'].min(),
        end=df['timestamp'].max(),
        freq='1H'
    )
    
    expected_count = len(expected_range)
    actual_count = len(df)
    missing_hours = expected_count - actual_count
    
    print(f"Expected hourly records: {expected_count:,}")
    print(f"Actual records: {actual_count:,}")
    print(f"Missing hours: {missing_hours:,} ({missing_hours/expected_count*100:.2f}%)")
    
    if missing_hours > 0:
        print("\n⚠ Temporal gaps detected - will reindex and interpolate")
        
        # Reindex to complete hourly series
        df = df.set_index('timestamp').reindex(expected_range).rename_axis('timestamp').reset_index()
        print(f"✓ Reindexed to {len(df):,} hourly records")
    else:
        print("\n✓ No temporal gaps - data is complete!")
else:
    print("⚠ Cannot check for temporal gaps without timestamp column")

In [ ]:
# Flag interpolated rows (before interpolation)
df['is_interpolated'] = df[['open', 'high', 'low', 'close']].isnull().any(axis=1).astype(int)
interpolated_count = df['is_interpolated'].sum()
print(f"Rows requiring interpolation: {interpolated_count:,} ({interpolated_count/len(df)*100:.2f}%)")

In [ ]:
# Handle missing values by type
print("Applying interpolation strategy...\n")

# Price columns: Linear interpolation (continuous process assumption)
price_cols = ['open', 'high', 'low', 'close']
for col in price_cols:
    if col in df.columns:
        before_nulls = df[col].isnull().sum()
        df[col] = df[col].interpolate(method='linear', limit_direction='both')
        after_nulls = df[col].isnull().sum()
        print(f"  • {col.upper()}: Interpolated {before_nulls - after_nulls} missing values")

# Volume & trade count: Zero-filling (no trading activity assumption)
volume_cols = ['volume', 'trade_count', 'taker_buy_volume', 'taker_buy_quote_volume']
for col in volume_cols:
    if col in df.columns:
        before_nulls = df[col].isnull().sum()
        df[col] = df[col].fillna(0)
        after_nulls = df[col].isnull().sum()
        if before_nulls > 0:
            print(f"  • {col.upper()}: Zero-filled {before_nulls - after_nulls} missing values")

print("\n✓ Interpolation complete")
print(f"✓ Remaining nulls: {df.isnull().sum().sum()}")

In [ ]:
# Validate price consistency (High >= Low, High >= Open/Close, Low <= Open/Close)
print("Validating OHLC price consistency...\n")

issues = []

# Check 1: High >= Low
invalid_hl = (df['high'] < df['low']).sum()
if invalid_hl > 0:
    issues.append(f"⚠ {invalid_hl} rows where High < Low")
else:
    print("✓ All High >= Low")

# Check 2: High >= Open and Close
invalid_ho = (df['high'] < df['open']).sum()
invalid_hc = (df['high'] < df['close']).sum()
if invalid_ho > 0 or invalid_hc > 0:
    issues.append(f"⚠ High < Open: {invalid_ho}, High < Close: {invalid_hc}")
else:
    print("✓ All High >= Open and Close")

# Check 3: Low <= Open and Close
invalid_lo = (df['low'] > df['open']).sum()
invalid_lc = (df['low'] > df['close']).sum()
if invalid_lo > 0 or invalid_lc > 0:
    issues.append(f"⚠ Low > Open: {invalid_lo}, Low > Close: {invalid_lc}")
else:
    print("✓ All Low <= Open and Close")

# Check 4: No zero or negative prices
invalid_prices = ((df[price_cols] <= 0).sum(axis=1) > 0).sum()
if invalid_prices > 0:
    issues.append(f"⚠ {invalid_prices} rows with zero or negative prices")
else:
    print("✓ No zero or negative prices")

if issues:
    print("\n" + "\n".join(issues))
    print("\nNote: These anomalies may be due to flash crashes or data errors.")
    print("Consider manual inspection or outlier removal if count is significant.")
else:
    print("\n✓ All OHLC validations passed!")

---
## 3. Garman-Klass Volatility Implementation

In [ ]:
def garman_klass_volatility(open_price, high_price, low_price, close_price):
    """
    Compute Garman-Klass (1980) volatility estimator.
    
    Formula:
    GK = sqrt(0.5 * [log(H/L)]^2 - (2*log(2) - 1) * [log(C/O)]^2)
    
    Parameters:
    -----------
    open_price : float
        Opening price of interval
    high_price : float
        Highest price during interval
    low_price : float
        Lowest price during interval
    close_price : float
        Closing price of interval
    
    Returns:
    --------
    float : Garman-Klass volatility estimate (returns NaN for invalid prices)
    
    Reference:
    ----------
    Garman, M. B., & Klass, M. J. (1980). On the estimation of security price 
    volatilities from historical data. Journal of Business, 53(1), 67-78.
    """
    # Validate inputs
    if any(pd.isna([open_price, high_price, low_price, close_price])):
        return np.nan
    
    if any(p <= 0 for p in [open_price, high_price, low_price, close_price]):
        return np.nan
    
    # Compute log ratios
    hl_ratio = np.log(high_price / low_price)
    co_ratio = np.log(close_price / open_price)
    
    # Garman-Klass formula
    gk = np.sqrt(
        0.5 * hl_ratio**2 - (2 * np.log(2) - 1) * co_ratio**2
    )
    
    return gk

print("✓ Garman-Klass function defined")
print("\nFunction signature: garman_klass_volatility(open, high, low, close)")

In [ ]:
# Apply Garman-Klass formula to entire dataset
print("Computing GK volatility for all hourly intervals...\n")

df['gk_volatility'] = df.apply(
    lambda row: garman_klass_volatility(
        row['open'], 
        row['high'], 
        row['low'], 
        row['close']
    ),
    axis=1
)

# Check for NaN values
gk_nulls = df['gk_volatility'].isnull().sum()
print(f"✓ GK volatility computed for {len(df) - gk_nulls:,} intervals")
if gk_nulls > 0:
    print(f"⚠ {gk_nulls} NaN values (likely due to invalid prices)")
    print("  These will be handled in feature engineering phase")

In [ ]:
# Descriptive statistics for GK volatility
print("Garman-Klass Volatility Statistics:")
print("=" * 60)
print(df['gk_volatility'].describe())

# Identify potential outliers (>3 std deviations)
gk_mean = df['gk_volatility'].mean()
gk_std = df['gk_volatility'].std()
outlier_threshold = gk_mean + 3 * gk_std
outliers = (df['gk_volatility'] > outlier_threshold).sum()

print(f"\nPotential outliers (>3σ): {outliers} ({outliers/len(df)*100:.3f}%)")
print(f"Outlier threshold: {outlier_threshold:.6f}")

In [ ]:
# Visualize GK volatility distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(df['gk_volatility'].dropna(), bins=100, edgecolor='black', alpha=0.7)
axes[0].axvline(gk_mean, color='red', linestyle='--', linewidth=2, label=f'Mean: {gk_mean:.6f}')
axes[0].axvline(outlier_threshold, color='orange', linestyle='--', linewidth=2, label=f'3σ: {outlier_threshold:.6f}')
axes[0].set_xlabel('GK Volatility', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Distribution of Garman-Klass Volatility', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Time series (downsampled for visibility if >10k points)
if 'timestamp' in df.columns:
    plot_df = df[['timestamp', 'gk_volatility']].dropna()
    if len(plot_df) > 10000:
        plot_df = plot_df.iloc[::10]  # Plot every 10th point
    axes[1].plot(plot_df['timestamp'], plot_df['gk_volatility'], linewidth=0.5, alpha=0.7)
    axes[1].axhline(gk_mean, color='red', linestyle='--', linewidth=1, label='Mean')
    axes[1].set_xlabel('Time', fontsize=12)
    axes[1].set_ylabel('GK Volatility', fontsize=12)
    axes[1].set_title('GK Volatility Over Time', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(alpha=0.3)
else:
    axes[1].text(0.5, 0.5, 'Timestamp not available', ha='center', va='center', fontsize=14)
    axes[1].axis('off')

plt.tight_layout()
plt.show()

print("✓ Volatility visualizations generated")

---
## 4. Target Variable Creation

In [ ]:
# Create target: next hour's GK volatility
print("Creating target variable: next-hour GK volatility\n")

# Shift GK volatility backward by 1 (shift=-1 means look ahead)
df['target_gk_next_hour'] = df['gk_volatility'].shift(-1)

# Count valid targets
valid_targets = df['target_gk_next_hour'].notna().sum()
print(f"✓ Target variable created: {valid_targets:,} valid samples")
print(f"  (Last row has no future target, as expected)")

# Remove last row (no future target)
df_original_len = len(df)
df = df[:-1].copy()
print(f"\n✓ Dropped last row: {df_original_len:,} → {len(df):,} rows")

In [ ]:
# Verify no data leakage: current GK should differ from target
print("Data Leakage Check:")
print("=" * 60)

# Correlation between current and next-hour volatility
correlation = df[['gk_volatility', 'target_gk_next_hour']].corr().iloc[0, 1]
print(f"Correlation(GK_current, GK_next_hour): {correlation:.4f}")

if correlation > 0.95:
    print("⚠ Very high correlation - may indicate data leakage or trivial prediction")
elif correlation > 0.5:
    print("✓ Moderate correlation - volatility clustering detected (expected in finance)")
else:
    print("✓ Low correlation - prediction task is non-trivial")

# Check if any rows are identical
identical_rows = (df['gk_volatility'] == df['target_gk_next_hour']).sum()
print(f"\nRows where current GK == next GK: {identical_rows} ({identical_rows/len(df)*100:.2f}%)")
print("  (Some matches are expected in stable market conditions)")

In [ ]:
# Visualize target distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Target distribution
axes[0].hist(df['target_gk_next_hour'].dropna(), bins=100, edgecolor='black', alpha=0.7, color='green')
axes[0].set_xlabel('Target: Next-Hour GK Volatility', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Distribution of Target Variable', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)

# Scatter: current vs next-hour volatility
sample_size = min(5000, len(df))  # Downsample for visibility
sample_df = df.sample(n=sample_size, random_state=42)
axes[1].scatter(sample_df['gk_volatility'], sample_df['target_gk_next_hour'], 
                alpha=0.3, s=10, color='blue')
axes[1].plot([df['gk_volatility'].min(), df['gk_volatility'].max()],
             [df['gk_volatility'].min(), df['gk_volatility'].max()],
             'r--', linewidth=2, label='y=x (perfect correlation)')
axes[1].set_xlabel('Current GK Volatility', fontsize=12)
axes[1].set_ylabel('Next-Hour GK Volatility', fontsize=12)
axes[1].set_title(f'Current vs Next-Hour Volatility (n={sample_size:,})', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Target variable visualizations generated")

---
## 5. Final Data Quality Report

In [ ]:
print("\n" + "="*70)
print("PHASE 1 COMPLETION REPORT: DATA CLEANING & GROUND TRUTH")
print("="*70)

print(f"\n📊 Dataset Statistics:")
print(f"  • Total hourly records: {len(df):,}")
if 'timestamp' in df.columns:
    print(f"  • Time span: {df['timestamp'].min()} to {df['timestamp'].max()}")
    total_days = (df['timestamp'].max() - df['timestamp'].min()).days
    print(f"  • Duration: {total_days:,} days ({total_days/365.25:.1f} years)")

print(f"\n🔧 Data Cleaning:")
print(f"  • Interpolated rows: {df['is_interpolated'].sum():,} ({df['is_interpolated'].mean()*100:.2f}%)")
print(f"  • Remaining nulls: {df.isnull().sum().sum()}")

print(f"\n📈 Garman-Klass Volatility:")
print(f"  • Valid GK calculations: {df['gk_volatility'].notna().sum():,}")
print(f"  • Mean volatility: {df['gk_volatility'].mean():.6f}")
print(f"  • Std volatility: {df['gk_volatility'].std():.6f}")
print(f"  • Min volatility: {df['gk_volatility'].min():.6f}")
print(f"  • Max volatility: {df['gk_volatility'].max():.6f}")

print(f"\n🎯 Target Variable:")
print(f"  • Valid targets: {df['target_gk_next_hour'].notna().sum():,}")
print(f"  • Target mean: {df['target_gk_next_hour'].mean():.6f}")
print(f"  • Autocorrelation: {correlation:.4f}")

print(f"\n💾 Output Columns: {len(df.columns)}")
print(f"  {df.columns.tolist()}")

print("\n" + "="*70)
print("✓ PHASE 1 COMPLETE - Ready for Feature Engineering")
print("="*70)

---
## 6. Save Cleaned Dataset

In [ ]:
# Save to CSV
OUTPUT_FILE = Path('cleaned_data_with_gk_target.csv')
df.to_csv(OUTPUT_FILE, index=False)

print(f"✓ Cleaned dataset saved to: {OUTPUT_FILE}")
print(f"✓ File size: {OUTPUT_FILE.stat().st_size / (1024**2):.2f} MB")
print(f"\nNext step: Open 2_feature_engineering.ipynb to create 50+ candidate features")

In [ ]:
# Quick preview of saved file structure
print("\nFinal dataset preview (first 5 rows):")
df.head()